지역
- 동두천시
- D군, 수요 데이터 없음

중심점
- 아파트

고려한 요소들
- 공영주차장
- 쏘카존
- 지하철
- 아파트

스케일링 여부
- 평균으로 나눔

In [183]:
import pandas as pd
import numpy as np
import dataframe_image as dfi

In [184]:
# 데이터프레임을 그림 파일로 저장
save_option = 1 # 1 : dfi / 2 : df2img / 3 : do nothing
import df2img
def save(df, path):
    if save_option == 1:
        dfi.export(df, path)
    if save_option == 2:
        df2img.save_dataframe(fig=df2img.plot_dataframe(df), filename=path)
    if save_option == 3:
        pass

각 군집별 도시들
- cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '시흥시', '용인시 기흥구', '의정부시', '파주시']
- cluster B : ['구리시', '군포시', '성남시 수정구', '성남시 중원구', '수원시 장안구', '수원시 팔달구', '안성시', '안양시 만안구', '양주시', '오산시', '용인시 처인구', '의왕시', '이천시']
- cluster C : ['남양주시', '부천시', '화성시']
- cluster D : ['가평군', '과천시', '동두천시', '양평군', '여주시', '연천군', '포천시']
- cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '광주시', '수원시 권선구', '수원시 영통구', '안산시 단원구', '안산시 상록구', '안양시 동안구', '용인시 수지구', '하남시']
- cluster F : ['평택시']

각 군집별 (수요 데이터가 있는) 도시들
- Cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '의정부시']
- Cluster B : ['구리시', '성남시 수정구', '성남시 중원구', '안양시 만안구']
- Cluster C : ['남양주시', '화성시']
- Cluster D : ['양평군']
- Cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '안양시 동안구', '하남시']
- Cluster F : []

동두천시는 D군에 속해있고, 수요데이터는 없는 도시이다.

# 데이터 작업

## 행복주택

In [185]:
happys = pd.read_csv("../../data/data_b_raw/2b206_0111_경기도 행복주택주거지현황.csv")
happys.rename(columns={'주소':'행복주택 주소'}, inplace=True)
happys = happys[happys['지역']=='동두천시'].reset_index(drop=True)
happy_names = happys[['행복주택 주소']]
happys = happys[['경도', '위도']].rename(columns={'경도':'x', '위도':'y'})
happys = happys.values

## 아파트

In [186]:
apartments = pd.read_csv("../../data/data_b_raw/2b209_0202_경기도 동두천시_공동주택현황_20220624.csv", encoding='CP949')
apartments.rename(columns={'도로명 주소':'아파트 주소', '아파트명':'아파트 이름'}, inplace=True)
apartment_names = apartments[['아파트 이름', '아파트 주소']]
apartments = apartments[['경도','위도']]
apartments.rename(columns={'경도':'x', '위도':'y'}, inplace=True)
apartments = apartments.values
apartments.shape

(41, 2)

## 공영주차장

In [187]:
parks = pd.read_csv("../../data/data_b_raw/2b208_0202_한국교통안전공단_전국공영주차장정보_20191224.csv", encoding='CP949')
parks.rename(columns={'주차장지번주소':'주차장 주소'}, inplace=True)
parks = parks[parks['지역구분']=='경기도']
parks = parks[parks['지역구분_sub']=='동두천시'].reset_index(drop=True)
park_names = parks[['주차장 주소']]
parks = parks[['경도', '위도']].rename(columns={'경도':'x','위도':'y'})
parks = parks.values
parks.shape

(95, 2)

## 쏘카존

In [188]:
# 쏘카존 갯수 데이터(김선중)
# 출처 : https://d3s0tskafalll9.cloudfront.net/media/documents/20210823_socar_zone.csv (going deeper 4-6)
zones = pd.read_csv("../../data/data_b_raw/2b201_1228_20210823_socar_zone.csv")
zones.rename(columns={'zone_name':'존 이름'}, inplace=True)
zones = zones[zones['region2']=='동두천시'].reset_index(drop=True)
zone_names = zones[['존 이름']]
zones = zones[['lng','lat']]
zones.rename(columns={'lng':'x','lat':'y'}, inplace=True)
zones = zones.values
zones.shape

(4, 2)

## 지하철역

In [189]:
subways = pd.read_csv("../../data/data_b_raw/2b211_0202_dongducheon_subways.csv")
subway_names = subways[['역 이름','역 주소']]
subways = subways[['경도', '위도']].rename(columns={'위도':'y','경도':'x'})
subways = subways[['x', 'y']].values
subways.shape

(5, 2)

In [190]:
subway_names

,역 이름,역 주소
0,지행역,경기도 동두천시 평화로 2285
1,동두천중앙역,경기도 동두천시 동두천로 228
2,보산역,경기도 동두천시 평화로 2539
3,동두천역,경기도 동두천시 평화로 2687
4,소요산역,경기도 동두천시 평화로 2925


# 제안

In [191]:
n_apartments = len(apartments)
n_parks = len(parks)
n_zones = len(zones)
n_subways = len(subways)
print("n_apartments :",n_apartments)
print("n_parks :",n_parks)
print("n_zones :",n_zones)
print("n_subways :",n_subways)

n_apartments : 41
n_parks : 95
n_zones : 4
n_subways : 5


In [192]:
# euclidean distance
def d(x:np.array, y:np.array) -> float:
    return np.linalg.norm(x-y)
x = np.array([0,3])
y = np.array([4,0])
d(x,y)

5.0

## score 설정

임의의 행복주택 $H$에 대하여, 다음과 같은 사실들을 가정한다.
- 가정1 : $H$에서 공영주차장이 가까이 위치할 수록 $H$의 근처에 쏘카존을 제안할 만하다. (subscript : P)
- 가정2 : $H$에서 쏘카존이 멀리 위치할 수록 $H$의 근처에 쏘카존을 제안할 만하다. (subscript : Z)
- 가정3 : $H$에서 지하철역이 멀리 위치할 수록 $H$의 근처에 쏘카존을 제안할 만하다. (subscript : S)
(지하철이 있으면, 지하철을 타지 쏘카를 타지는 않을 것이다.)
- 가정3 : $H$의 근처에 아파트가 많을수록 $H$의 근처에 쏘카존을 제안할 만하다. (미반영) (subscript : A)

행복주택 $H$에 대하여, 해당 행복주택 근처에 쏘카존을 제안할 만한지를 나타내는 score (score function)를 $f(H)$라고 하자.
<!-- (일반적인 목적함수 e.g. loss function/cost function/error function는 목적함수의 값을 최소화하는 것이 목적이지만, 이번 목적함수는 최대화하는 것이 목적이다.) -->

\begin{align*}
f(H)&=\alpha_P f_P(H) + \alpha_Z f_Z(H) + \alpha_S f_S(H) + \alpha_A f_A(H)\\
f_P(H)&:-(\text{가장 가까운 공영주차장까지의 거리})\\
f_Z(H)&:(\text{가장 가까운 쏘카존까지의 거리})\\
f_S(H)&:(\text{가장 가까운 지하철역까지의 거리})\\
f_A(H)&:(\text{반경 1km 이내의 아파트의 개수})
\end{align*}

단, $\alpha_P$, $\alpha_Z$, $\alpha_S$, $\alpha_A$는 각각의 additive subfunction에 대한 가중치로, 해당 feature들을 어느 정도의 중요도로 반영할지를 나타내는 값이다.
가중치가 모두 1이면, score는 다음과 같은 비교적 간단한 식이 된다.

$$f(H) = -(\text{가장 가까운 공영주차장까지의 거리}) + (\text{가장 가까운 쏘카존까지의 거리}) + (\text{가장 가까운 지하철역까지의 거리}) + (\text{반경 1km 이내의 아파트의 개수})$$

## 계산

In [193]:
# 1 degree of lattitude = 111.1km
# 1 km = 1/111.1 degree of lattitude = 0.009 degree of lattitude
# 각 아파트 A 마다 A와의 거리가 0.009(1km) 이하인 아파트들의 개수를 구한다.

In [194]:
# distances : the distances of the apartment A and every apartment.
# cardinality : the number of apartments which is closer to apartment A than 0.009.
# park_info : the list of cardinalities for every apartment.
apartments_info = []
for apartment_ in apartments:
    distances = np.array([d(apartment, apartment_) for apartment in apartments])
    cardinality = sum(distances <=0.009)
    apartments_info.append(cardinality)
apartments_info = np.array(apartments_info)
apartments_info.shape

(41,)

In [195]:
apartment_names.columns

Index(['아파트 이름', '아파트 주소'], dtype='object')

In [196]:
apartments_info_ = pd.DataFrame({'근처 아파트 개수':apartments_info})
apartments_info_.reset_index(inplace=True)
apartments_info_['아파트 이름'] = apartments_info_['index'].replace(apartment_names['아파트 이름'].to_dict())
apartments_info_['아파트 주소'] = apartments_info_['index'].replace(apartment_names['아파트 주소'].to_dict())
apartments_info_.head()


,index,근처 아파트 개수,아파트 이름,아파트 주소
0,0,4,건영,경기도 동두천시 강변로296번길
1,1,9,기상,경기도 동두천시 못골로 60
2,2,15,대방노블랜드,경기도 동두천시 거북마루로 45
3,3,20,대방노블랜드1차,경기도 동두천시 삼육사로1002번길25
4,4,19,생연주공(3단지),경기도 동두천시 행선로 112


In [197]:
# distances : the distances of the apartments and every parking lot
# park_info : the list of (min,index) pair for every apartment.
parks_info = []
for apartment in apartments:
    distances = [d(apartment, park) for park in parks]
    parks_info.append(np.array([np.min(distances),np.argmin(distances)]))
parks_info = np.array(parks_info)
parks_info.shape

(41, 2)

In [198]:
parks_info_ = pd.DataFrame({'주차장 최소거리':parks_info[:,0], 'index':np.round(parks_info[:,1]).astype(int)})
parks_info_['주차장 주소'] = parks_info_['index'].replace(park_names['주차장 주소'].to_dict())
parks_info_.head()

,주차장 최소거리,index,주차장 주소
0,0.004769,26,경기도 동두천시 보산동 387-7
1,0.006555,13,"""경기도 동두천시 생연동732-4,5,6"""
2,0.001049,79,경기도 동두천시 생연동 714-9
3,0.001164,21,경기도 동두천시 생연동 385-13
4,0.001127,19,경기도 동두천시 생연동 348


In [199]:
# distances : the distances of the apartments and every zone
# zone_info : the list of (min,index) pair for every apartment.
zones_info = []
for apartment in apartments:
    distances = [d(apartment, zone) for zone in zones]
    zones_info.append(np.array([np.min(distances),np.argmin(distances)]))
zones_info = np.array(zones_info)
zones_info

array([[1.62862683e-02, 3.00000000e+00],
       [1.56577792e-02, 2.00000000e+00],
       [2.78811567e-03, 0.00000000e+00],
       [6.12653711e-03, 2.00000000e+00],
       [8.02069854e-03, 2.00000000e+00],
       [1.48408625e-03, 0.00000000e+00],
       [4.14848707e-03, 0.00000000e+00],
       [4.02763814e-03, 0.00000000e+00],
       [3.37479792e-03, 2.00000000e+00],
       [1.70228229e-03, 2.00000000e+00],
       [1.20071098e-02, 2.00000000e+00],
       [7.09629009e-03, 0.00000000e+00],
       [1.81129629e-02, 3.00000000e+00],
       [9.83067266e-03, 2.00000000e+00],
       [1.94102267e-03, 0.00000000e+00],
       [2.65591886e-03, 1.00000000e+00],
       [4.06536062e-03, 1.00000000e+00],
       [4.19202970e-03, 1.00000000e+00],
       [7.65344008e-03, 1.00000000e+00],
       [6.14375463e-03, 1.00000000e+00],
       [9.01957516e-03, 1.00000000e+00],
       [7.84649355e-03, 2.00000000e+00],
       [7.33793465e-03, 2.00000000e+00],
       [6.83273774e-03, 2.00000000e+00],
       [5.765167

In [200]:
zones_info_ = pd.DataFrame({'쏘카존 최소거리':zones_info[:,0], 'index':np.round(zones_info[:,1]).astype(int)})
zones_info_['존 이름'] = zones_info_['index'].replace(zone_names['존 이름'].to_dict())
zones_info_.head()

,쏘카존 최소거리,index,존 이름
0,0.016286,3,동두천역
1,0.015658,2,지행역 3번출구
2,0.002788,0,동두천 큰시장 앞
3,0.006127,2,지행역 3번출구
4,0.008021,2,지행역 3번출구


In [201]:
# distances : the distances of the apartments and every subway
# subway_info : the list of (min,index) pair for every apartments.
subways_info = []
for apartment in apartments:
    distances = [d(apartment, subway) for subway in subways]
    subways_info.append(np.array([np.min(distances),np.argmin(distances)]))
subways_info = np.array(subways_info)
subways_info

array([[1.90047314e-02, 3.00000000e+00],
       [1.94162956e-02, 0.00000000e+00],
       [3.65818774e-03, 1.00000000e+00],
       [1.03768726e-02, 0.00000000e+00],
       [1.18407716e-02, 0.00000000e+00],
       [3.34412448e-03, 1.00000000e+00],
       [2.58919843e-03, 1.00000000e+00],
       [8.15142258e-03, 1.00000000e+00],
       [7.67142086e-03, 1.00000000e+00],
       [5.05816956e-03, 0.00000000e+00],
       [1.55790720e-02, 0.00000000e+00],
       [3.39923459e-03, 2.00000000e+00],
       [1.89005213e-02, 2.00000000e+00],
       [1.37622053e-02, 0.00000000e+00],
       [5.16838752e-03, 1.00000000e+00],
       [6.50731155e-03, 1.00000000e+00],
       [5.04468742e-03, 1.00000000e+00],
       [1.81767102e-03, 0.00000000e+00],
       [6.15622165e-03, 0.00000000e+00],
       [2.87671043e-03, 0.00000000e+00],
       [5.42890862e-03, 0.00000000e+00],
       [1.12443055e-02, 0.00000000e+00],
       [1.08845483e-02, 0.00000000e+00],
       [1.09663527e-02, 0.00000000e+00],
       [9.743664

In [202]:
subways_info_ = pd.DataFrame({'지하철 최소거리':subways_info[:,0], 'index':np.round(subways_info[:,1]).astype(int)})
subways_info_['역 이름'] = subways_info_['index'].replace(subway_names['역 이름'].to_dict())
subways_info_['역 주소'] = subways_info_['index'].replace(subway_names['역 주소'].to_dict())
subways_info_.head()

,지하철 최소거리,index,역 이름,역 주소
0,0.019005,3,동두천역,경기도 동두천시 평화로 2687
1,0.019416,0,지행역,경기도 동두천시 평화로 2285
2,0.003658,1,동두천중앙역,경기도 동두천시 동두천로 228
3,0.010377,0,지행역,경기도 동두천시 평화로 2285
4,0.011841,0,지행역,경기도 동두천시 평화로 2285


In [203]:
parks_info_.drop('index', axis=1), zones_info_.drop('index', axis=1), subways_info_.drop('index', axis=1)

(    주차장 최소거리                   주차장 주소
 0   0.004769       경기도 동두천시 보산동 387-7
 1   0.006555  "경기도 동두천시 생연동732-4,5,6"
 2   0.001049       경기도 동두천시 생연동 714-9
 3   0.001164      경기도 동두천시 생연동 385-13
 4   0.001127         경기도 동두천시 생연동 348
 5   0.001674  경기도 동두천시 생연동 811-1외 3필지
 6   0.002804       경기도 동두천시 생연동 714-9
 7   0.003117      경기도 동두천시 생연동 693-16
 8   0.003127       경기도 동두천시 생연동 714-9
 9   0.002618         경기도 동두천시 지행동 720
 10  0.004856         경기도 동두천시 생연동 348
 11  0.001089        경기도 동두천시 상패동 66-4
 12  0.003906       경기도 동두천시 생연동 522-1
 13  0.002514         경기도 동두천시 생연동 348
 14  0.000808       경기도 동두천시 생연동 810-5
 15  0.002115       경기도 동두천시 생연동 714-9
 16  0.003651         경기도 동두천시 지행동 722
 17  0.001372      경기도 동두천시 송내동 699-14
 18  0.001460         경기도 동두천시 송내동 689
 19  0.003424      경기도 동두천시 송내동 699-14
 20  0.001384         경기도 동두천시 송내동 676
 21  0.002112         경기도 동두천시 생연동 348
 22  0.001713         경기도 동두천시 생연동 348
 23  0.000522         경기도 동두천시 생연동 348
 24  0.001748         경기도

In [204]:
scores = pd.concat([apartments_info_.drop('index',axis=1), parks_info_.drop('index', axis=1), zones_info_.drop('index', axis=1), subways_info_.drop('index', axis=1)], axis=1)
scores.head()

,근처 아파트 개수,아파트 이름,아파트 주소,주차장 최소거리,주차장 주소,쏘카존 최소거리,존 이름,지하철 최소거리,역 이름,역 주소
0,4,건영,경기도 동두천시 강변로296번길,0.004769,경기도 동두천시 보산동 387-7,0.016286,동두천역,0.019005,동두천역,경기도 동두천시 평화로 2687
1,9,기상,경기도 동두천시 못골로 60,0.006555,"""경기도 동두천시 생연동732-4,5,6""",0.015658,지행역 3번출구,0.019416,지행역,경기도 동두천시 평화로 2285
2,15,대방노블랜드,경기도 동두천시 거북마루로 45,0.001049,경기도 동두천시 생연동 714-9,0.002788,동두천 큰시장 앞,0.003658,동두천중앙역,경기도 동두천시 동두천로 228
3,20,대방노블랜드1차,경기도 동두천시 삼육사로1002번길25,0.001164,경기도 동두천시 생연동 385-13,0.006127,지행역 3번출구,0.010377,지행역,경기도 동두천시 평화로 2285
4,19,생연주공(3단지),경기도 동두천시 행선로 112,0.001127,경기도 동두천시 생연동 348,0.008021,지행역 3번출구,0.011841,지행역,경기도 동두천시 평화로 2285


In [205]:
weights = scores.describe().loc['mean'].values
# alpha_A = weights[0]
# alpha_P = weights[1]
# alpha_Z = weights[2]
# alpha_S = weights[3]
weights

array([1.45609756e+01, 2.62156868e-03, 6.90336362e-03, 8.63973005e-03])

In [206]:
scores['스코어'] = 1/weights[0]*(-scores['주차장 최소거리']) + 1/weights[1]*scores['쏘카존 최소거리'] + 1/weights[2]*scores['지하철 최소거리'] + 1/weights[3]*scores['근처 아파트 개수']
scores.sort_values(by='스코어', ascending=False, inplace=True)
scores


,근처 아파트 개수,아파트 이름,아파트 주소,주차장 최소거리,주차장 주소,쏘카존 최소거리,존 이름,지하철 최소거리,역 이름,역 주소,스코어
8,24,부영6단지(2차),경기도 동두천시 동두천로 171,0.003127,경기도 동두천시 생연동 714-9,0.003375,지행역 3번출구,0.007671,동두천중앙역,경기도 동두천시 동두천로 228,2780.262935
9,24,부영9단지(3차),경기도 동두천시 동두천로 137. 139,0.002618,경기도 동두천시 지행동 720,0.001702,지행역 3번출구,0.005058,지행역,경기도 동두천시 평화로 2285,2779.246439
7,23,부영5단지(5차),경기도 동두천시 동두천로 185,0.003117,경기도 동두천시 생연동 693-16,0.004028,동두천 큰시장 앞,0.008151,동두천중앙역,경기도 동두천시 동두천로 228,2664.837136
15,23,송내대방7/8,경기도 동두천시 거북마루로 64,0.002115,경기도 동두천시 생연동 714-9,0.002656,지행역 2번출구,0.006507,동두천중앙역,경기도 동두천시 동두천로 228,2664.075801
32,22,지행역휴먼빌,경기도 동두천시 행선로 52,0.004365,경기도 동두천시 지행동 482,0.004674,지행역 3번출구,0.006738,지행역,경기도 동두천시 평화로 2285,2549.134574
28,22,지행주공2단지,경기도 동두천시 평화로 2316-13,0.004912,"""경기도 동두천시 생연동 374-1,2""",0.002015,지행역 3번출구,0.006112,지행역,경기도 동두천시 평화로 2285,2548.029395
29,22,지행현대,경기도 동두천시 평화로 2316-20,0.005378,"""경기도 동두천시 생연동 374-1,2""",0.002063,지행역 3번출구,0.005742,지행역,경기도 동두천시 평화로 2285,2547.994110
22,21,에이스2단지,경기도 동두천시 행선로 100-9,0.001713,경기도 동두천시 생연동 348,0.007338,지행역 3번출구,0.010885,지행역,경기도 동두천시 평화로 2285,2435.007147
24,21,에이스4단지,경기도 동두천시 행선로 123-49,0.001748,경기도 동두천시 생연동 348,0.005765,지행역 3번출구,0.009744,지행역,경기도 동두천시 평화로 2285,2434.241945
38,21,브라운스톤,경기도 동두천시 장고갯로 23,0.001729,"""경기도 동두천시 생연동 374-1,2""",0.004319,지행역 3번출구,0.009155,지행역,경기도 동두천시 평화로 2285,2433.605096


In [207]:
save(scores, "results/1d202_0202_scores_동두천.png")